In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys


import tflearn.data_utils as du
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

DIRECTORY = r"C:\Users\rassa\Documents\Machine Learning A-Z  Udemy\Arabic Handwritten Characters Dataset"



Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#Load the dataset
#header = 0 because
#this creates a pandas dataframe (basically a table)
train_data = pd.read_csv(DIRECTORY +  '/csvTrainImages 13440x1024.csv', header = None)
train_label = pd.read_csv(DIRECTORY + '/csvTrainLabel 13440x1.csv', header = None)

test_label = pd.read_csv(DIRECTORY +  '/csvTestLabel 3360x1.csv', header = None)


#what happens if we take off header = none ?
#Row number(s) to use as the column names, and the start of the data. Default behavior is to infer the column names: if no 
#names are passed the behavior is identical to header=0 and column names are inferred from the first line of the file, 
#if column names are passed explicitly then the behavior is identical to header=None.
#notice if we do this we have one row less
test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv')
print(test_data)

test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv', header = None)
print(test_data)
type(train_data)

      0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.919  0.920  \
0     0    0    0    0    0    0    0    0    0    0  ...      0      0   
1     0    0    0    0    0    0    0    0    0    0  ...      0      0   
2     0    0    0    0    0    0    0    0    0    0  ...      0      0   
3     0    0    0    0    0    0    0    0    0    0  ...      0      0   
4     0    0    0    0    0    0    0    0    0    0  ...      0      0   
...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
3354  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3355  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3356  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3357  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3358  0    0    0    0    0    0    0    0    0    0  ...      0      0   

      0.921  0.922  0.923  0.924  0.925  0.926  0.927  0.928  
0         0      0      0      0    

pandas.core.frame.DataFrame

In [3]:
train_data = train_data.iloc[:,:].values.astype('float32')
train_label = train_label.iloc[:,:].values.astype('int32')-1
test_data = test_data.iloc[:,:].values.astype('float32')
test_label = test_label.iloc[:,:].values.astype('int32')-1


In [4]:
train_label = du.to_categorical(train_label,28)

In [5]:
train_data = train_data/255
test_data = test_data/255

In [6]:
train_data = train_data.reshape([-1, 32, 32, 1])
test_data = test_data.reshape([-1, 32, 32, 1])

In [7]:
#train_data, mean1 = du.featurewise_zero_center(train_data)
#test_data, mean2 = du.featurewise_zero_center(test_data)

In [8]:

recognizer = Sequential()
#for any kind of deep learning we use the sequential model in keras then 
#add layers to it
recognizer.add(Conv2D(filters = 18, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (32,32,1)))
#the first 2 dimensional convolusion layer will have 32 filters. filters or kernels are 
#what we use to extract features. in this case they are of size 5x5 (kernel size)
#in the first layer it is necessary to specify the shape so for our case its a
#32x32 pixel image and since it's black and white then it has only one dimension
#if it was colored then we would have (32,32,3)
#activation='relu' this is rectified linear unit. the output filters or convolced layers
#might contain some negative values so we apply the rectifier function (or other functions)
recognizer.add(Conv2D(filters = 18, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2)))
recognizer.add(Dropout(0.25))


recognizer.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
recognizer.add(Dropout(0.25))


recognizer.add(Flatten())
recognizer.add(Dense(units = 256, input_dim = 1024, activation = 'relu'))
recognizer.add(Dense(units = 256, activation = "relu"))
recognizer.add(Dropout(0.5))
recognizer.add(Dense(28, activation = "softmax"))

In [9]:
recognizer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 18)        180       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 18)        2934      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 18)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 18)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        5216      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0

In [10]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [11]:
recognizer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [12]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

In [13]:
datagen.fit(train_data)

In [14]:
recognizer.fit_generator(datagen.flow(train_data,train_label, batch_size=100),
                             epochs = 30, verbose = 2, steps_per_epoch=train_data.shape[0] // 100)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`model.fit_generator` is deprecated and '


Epoch 1/30
134/134 - 19s - loss: 2.3795 - acc: 0.2697
Epoch 2/30
134/134 - 18s - loss: 1.3553 - acc: 0.5394
Epoch 3/30
134/134 - 19s - loss: 0.9788 - acc: 0.6685
Epoch 4/30
134/134 - 19s - loss: 0.7749 - acc: 0.7329
Epoch 5/30
134/134 - 21s - loss: 0.6438 - acc: 0.7848
Epoch 6/30
134/134 - 19s - loss: 0.5268 - acc: 0.8205
Epoch 7/30
134/134 - 19s - loss: 0.4824 - acc: 0.8422
Epoch 8/30
134/134 - 20s - loss: 0.4081 - acc: 0.8639
Epoch 9/30
134/134 - 21s - loss: 0.3774 - acc: 0.8768
Epoch 10/30
134/134 - 21s - loss: 0.3439 - acc: 0.8888
Epoch 11/30
134/134 - 20s - loss: 0.3239 - acc: 0.8954
Epoch 12/30
134/134 - 21s - loss: 0.2917 - acc: 0.9056
Epoch 13/30
134/134 - 20s - loss: 0.2706 - acc: 0.9148
Epoch 14/30
134/134 - 20s - loss: 0.2682 - acc: 0.9149
Epoch 15/30
134/134 - 21s - loss: 0.2421 - acc: 0.9230
Epoch 16/30
134/134 - 24s - loss: 0.2340 - acc: 0.9238
Epoch 17/30
134/134 - 27s - loss: 0.2295 - acc: 0.9279
Epoch 18/30
134/134 - 26s - loss: 0.2223 - acc: 0.9305
Epoch 19/30
134/134

In [15]:
predictions = recognizer.predict(test_data)
predictions = np.argmax(predictions,axis = 1)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [16]:
cm = confusion_matrix(test_label, predictions)

In [17]:
accuracy = sum(cm[i][i] for i in range(28)) / test_label.shape[0]
print("accuracy = " + str(accuracy))

accuracy = 0.9755952380952381


In [18]:
print(cm)

[[119   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   1   0   0   0   0]
 [  0 118   1   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0 117   2   0   0   0   0   0   0   0   1   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   3 117   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 120   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1 118   0   0   0   0   0   0   0   0   0   0   0   1
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   5 115   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 116   1   2   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   2 113   0   2   0   0   0   0   